Preprocessing

In [0]:
import os
import numpy as np
import cv2 as cv
import h5py
import matplotlib.pyplot as plt

In [0]:
vidPath=r"/content/train.mp4"
imgStoragePath=r"/content/IMG"

IMGList=[]
def write_frames():
    op_flows=[]
    frames=[]
    count=0
    vidCap=cv.VideoCapture(r"/content/train.mp4")
    success=True
    while success:
        if(count%1000==0) and count >0:
            print(count)
        success,frame1=vidCap.read()
        if (success == True):
            frame1=cv.cvtColor(frame1,cv.COLOR_BGR2RGB)
            frame1=cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
            name=imgStoragePath
            filename=str(count)+r".jpg"
            IMGList.append(os.path.join(name, filename))
            cv.imwrite(os.path.join(name, filename),frame1)  
            count+=1
        else:
            print(count)
            print("video reading completed")
            continue
    return count

Total_frames=write_frames()

In [0]:
print(Total_frames,' :Total Frames')

print('Size of FileList: ',len(IMGList))

In [0]:
hdf5_path = r"/content/train.hdf5"
resize_size=(240, 320, 2)
resize_frame_size=(240, 320, 3)
'''Only use to reset dataset'''
with h5py.File(hdf5_path, "w") as f:
    f.create_dataset("frames", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
                    maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
                    chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
    f.create_dataset("speeds", shape = (1,1), maxshape = (None,1))
f.close()

In [0]:
def write_hdf5(hdf5_path, frames, speeds):
    with h5py.File(hdf5_path) as f:
        print(len(frames), len(speeds))
        print(f["frames"], f["speeds"])
        f["frames"].resize(f["frames"].len() + len(frames), axis = 0)
        f["speeds"].resize(f["speeds"].len() + len(speeds), axis = 0)
        f["frames"][-len(frames):] = frames
        f["speeds"][-len(speeds):] = speeds

In [0]:
def opticalFlowDense(image_current,image_next):
    image_current=np.array(image_current)
    image_next=np.array(image_current)
    
    #convert to grayscale
    gray_current=cv.cvtColor(image_current,cv.COLOR_RGB2GRAY)
    gray_next=cv.cvtColor(image_next,cv.COLOR_RGB2GRAY)
    flow=cv.calcOpticalFlowFarneback(gray_current,gray_next,None,0.5,1,15,2,5,1.3,0)
    return flow


def flipimage(image_current, image_next):
    image_current=cv.flip(image_current, 1)
    image_next=cv.flip(image_next,1)
    return image_current,image_next

def augment(image_current):
    brightness=np.random.uniform(0.5,1.5)
    image_current_1 = cv.cvtColor(image_current,cv.COLOR_RGB2HSV)
    image_current_1[:,:,2] = image_current_1[:,:,2]*brightness
    
    image_current_1 = cv.cvtColor(image_current_1,cv.COLOR_HSV2RGB)
    
    return image_current_1

def flipimage(image_current):
    image_current=cv.flip(image_current, 1)
    return image_current

def augment(image_current, image_next):
    brightness=np.random.uniform(0.5,1.5)
    image_current_1 = cv.cvtColor(image_current,cv.COLOR_RGB2HSV)
    image_current_1[:,:,2] = image_current_1[:,:,2]*brightness
    
    image_next_1 = cv.cvtColor(image_next,cv.COLOR_RGB2HSV)
    image_next_1[:,:,2] = image_next_1[:,:,2]*brightness
    
    image_current_1 = cv.cvtColor(image_current_1,cv.COLOR_HSV2RGB)
    image_next_1 = cv.cvtColor(image_next_1,cv.COLOR_HSV2RGB)
    
    return image_current_1,image_next_1


In [0]:
import csv

speeds=[]
with open(r"/content/train.txt") as csvfile:
    reader=csv.reader(csvfile)
    for speed in reader:
        speeds.append(speed)

#remove first elem
print(len(speeds))

In [0]:
Sequence_size=len(speeds)
print(Sequence_size, ': Sequence Size')

Batch_size=500
print(Batch_size,' : BatchSize')

Num_batches=int(np.ceil(Sequence_size/Batch_size))
print(Num_batches,' : Number of Batches')


All_indices=[item for item in range(1,Sequence_size)]
print(All_indices[-1],'Last elem')
print(All_indices[0],'First elem')


#print(All_indices,'All indices')

def generate_indices(Batch_num):
    return All_indices[Batch_num*Batch_size:(Batch_num+1)*Batch_size]

print(generate_indices(203))

print(generate_indices(0))

In [0]:
def make_op_flow_video():
    count=0
    for batch in range(0,Num_batches):
        #print(generate_indices(batch))
        indexes=generate_indices(batch)
        frames=[]
        speed=[]
        for index in indexes:
            #print([indexes[i],indexes[i+1]])
            
            if(count%1000==0) and count >0:
                print(count)
            
            frame2=cv.imread(IMGList[index])
            frame2=cv.cvtColor(frame2,cv.COLOR_BGR2RGB)
            
            
            frames.append(np.array(frame2))
            speed.append(speeds[index])
            count+=1
        
        frames=np.array(frames,dtype=np.uint8)
        speed=np.array(speed)
        speed=speed.astype(float)
        write_hdf5(hdf5_path, frames, speed)
    
    print("Completed Video Processing")
    return count        

In [0]:
print("Starting")

print("Nothing")
count=make_op_flow_video()
print("Complete: ",count)

print('check')

In [0]:
op_hdf5_path = r"/content/op1.hdf5"
with h5py.File(op_hdf5_path, "w") as f:
    f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
                    maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
                    chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
f.close()

In [0]:
def write_op_hdf5(hdf5_path, op_flows):
    with h5py.File(hdf5_path) as f:
        print( len(op_flows))
        print( f["op_flow"])
        f["op_flow"].resize(f["op_flow"].len() + len(op_flows), axis = 0)
        f["op_flow"][-len(op_flows):] = op_flows

def opticalFlowDense(image_current, image_next):
    """
    Args:
        image_current : RGB image
        image_next : RGB image
    return:
        optical flow magnitude and angle and stacked in a matrix
    """
    image_current = np.array(image_current)
    image_next = np.array(image_next)
    gray_current = cv.cvtColor(image_current, cv.COLOR_RGB2GRAY)
    gray_next = cv.cvtColor(image_next, cv.COLOR_RGB2GRAY)
    flow = cv.calcOpticalFlowFarneback(gray_current, gray_next, None, 0.5, 1, 15, 2, 5, 1.3, 0)
    return flow

def dense_op():
    op_flows = []
    count = 0
    vidcap = cv.VideoCapture(vidPath)
    success,frame1 = vidcap.read()
    frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
    frame1 = cv.resize(frame1, (0,0), fx=0.5, fy=0.5) 
    while success:
        if (count % 100 == 0) and count > 0:
            print(count)
        success,frame2 = vidcap.read()
        if success == True:
            frame2 = cv.cvtColor(frame2, cv.COLOR_BGR2RGB)
            frame2 = cv.resize(frame2, (0,0), fx=0.5, fy=0.5) 
            frame1,frame2=augment(frame1,frame2)
            flow = opticalFlowDense(frame1, frame2)
            op_flows.append(flow)
            frame1 = frame2
            count+=1
        else:
            print("video reading completed")
            continue
    
    write_op_hdf5(op_hdf5_path,np.array(op_flows))
    return count
dense_op()



End of Prepeocessing

In [0]:
import os
import numpy as np
import cv2 as cv
import h5py
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, MaxPool2D, CuDNNGRU, GlobalMaxPool2D, Reshape, GRU, \
concatenate, Input, TimeDistributed , Dense, BatchNormalization, SpatialDropout2D, SpatialDropout1D, Dropout, GlobalAvgPool2D, Flatten
from keras import Model
from keras.applications import Xception
import keras.backend as k
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Cropping2D
from keras.layers import Convolution2D, MaxPooling2D, Dropout,ConvLSTM2D,TimeDistributed
from keras.layers import Conv2D, MaxPool2D, CuDNNGRU, GlobalMaxPool2D, Reshape, GRU, \
concatenate, Input, TimeDistributed, Dense, BatchNormalization, SpatialDropout2D, SpatialDropout1D, Dropout, GlobalAvgPool2D, Flatten
from keras.layers.core import Dense,Activation,Flatten,Lambda
from keras.layers import Lambda
from math import ceil
from keras import optimizers
from keras.layers import LSTM
from keras.layers import CuDNNGRU
from keras.layers import ELU
from sklearn.model_selection import train_test_split, KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error


In [0]:
import glob 

hdf5_paths =  glob.glob(r"/content/train*.hdf5")
op_hdf5_paths =  glob.glob(r"/content/op*.hdf5")
augFactor=4

hdf5_paths.sort()
op_hdf5_paths.sort()

print(hdf5_paths)
print(op_hdf5_paths)

In [0]:
for path in op_hdf5_paths:
      with h5py.File(path, "r") as g:
        print("mean intensity of optical flow:" , g["op_flow"][1].mean())
        print("mean intensity of optical flow:" , g["op_flow"][0].mean())
        print("mean intensity of optical flow:" , g["op_flow"][20339].mean())
        print((g["op_flow"]).shape,'opFlows size')
    
for path in hdf5_paths:
      with h5py.File(path, "r") as f:
        print("mean intensity of optical flow:", f["frames"][5601].mean())
        plt.imshow(f["frames"][5601]/255)
        data_size=f["speeds"].shape[0]
        print((f["frames"]).shape,'frame size')
  

train_size=int(data_size)
data_size*=augFactor

print(train_size,'number of speed data samples')
print(data_size,'real Training Size')

## Developing a time history model


In [0]:
from statistics import median as median

def generate_indices(time_history,train_size):
# All we need to do is store the indices
    FrameIndices=[]
    SpeedIndices=[]
    
    print(train_size)
    for j in range(0,train_size-time_history+1):
        tempFrame=[]
        for i in range(0,time_history):
            tempFrame.append(i+j)
        if time_history==2:            
            SpeedIndices.append(min(tempFrame))
        else:
            SpeedIndices.append(median(tempFrame))
        FrameIndices.append(tempFrame)
    return FrameIndices,SpeedIndices

In [0]:
time_history = 2

train_size=int(data_size/augFactor)
FrameIndices,SpeedIndices=generate_indices(time_history,train_size)
train_size=len(FrameIndices)
print(train_size,'Revised Train Size')

train_size=len(SpeedIndices)
print(train_size,'Revised Train Size Again')

print(FrameIndices[-1],'last Frame slice')
print(SpeedIndices[-1], 'last Speed slice')

print()
print('Checking if within indices:')
for path in hdf5_paths:
    with h5py.File(path, "r") as f:
        plt.imshow(f["frames"][FrameIndices[-1][-1]]/255)
        print(f["speeds"][SpeedIndices[-1]],'Last Speed Indice Value')

for path in op_hdf5_paths:
    with h5py.File(path, "r") as f:
           print("mean intensity of optical flow:" , f["op_flow"][FrameIndices[-1][-1]].mean())

## Generator Class

In [0]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, hdf5_paths,op_hdf5_paths,FrameIndices,SpeedIndices,train_size,indexes = None, validation_mode = False):
        
        self.hdf5_paths = hdf5_paths
        self.op_hdf5_paths=op_hdf5_paths
        self.FrameIndices=FrameIndices
        self.SpeedIndices=SpeedIndices
        
        
        if indexes is None:
            with indexes is None:
                 with h5py.File(self.hdf5_path, "r") as f:
                        self.indexes=np.arrange(train_size)
        else:
            self.indexes=indexes
        self.batch_size=batch_size
        self.validation_mode=validation_mode
        if self.validation_mode==False:
            print("shuffling")
            self.on_epoch_end()
       
        
            
    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.ceil(len(self.indexes)/self.batch_size))
    
    def __getitem__(self,index):
        # Generate indexes of the batch
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate the data
        return self.__data_generation(indexes)
    
    def on_epoch_end(self):
        # updates indexes after each epoch
        if self.validation_mode == False:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self,indexes):
        # Generates data containing batch samples
        indexes=list(indexes)
        indexes.sort()## We are making the Generator Class to read the video and the speed file
        speeds=[]
        op_flow=[]
        
        for paths in self.hdf5_paths:
            with h5py.File(paths, "r") as f:
                for index in indexes:
                    #print(index,",",end="")
                    speeds.append(f["speeds"][self.SpeedIndices[index]])
                       
        
        for paths in self.op_hdf5_paths:
            with h5py.File(paths, "r") as f:
                for index in indexes:
                    op_flow.append(f["op_flow"][self.FrameIndices[index]])
                    i=(random.randint(0,8))
                    
        op_flow=np.array(op_flow)
        speeds=np.array(speeds)
        return [op_flow,speeds]

In [0]:
def build_model_history(history_size):
    
    op_flow_inp=Input(shape=(history_size,240,320,2))
    filter_size = (3,3)
   
    
    op_flow=TimeDistributed(Lambda(lambda x: (x / 255.0) - 0.5))(op_flow_inp)
    
    op_flow = TimeDistributed(BatchNormalization())(op_flow_inp)
    op_flow = TimeDistributed(Dropout(.5))(op_flow)
    op_flow = TimeDistributed(Conv2D(4, filter_size, activation = "relu", data_format = "channels_last"))(op_flow)
    op_flow = TimeDistributed(MaxPool2D())(op_flow)
    op_flow = TimeDistributed(Conv2D(8, filter_size, activation = "relu", data_format = "channels_last"))(op_flow)
    op_flow = TimeDistributed(MaxPool2D())(op_flow)
    op_flow = TimeDistributed(Conv2D(32, filter_size, activation = "relu", data_format = "channels_last"))(op_flow)
    op_flow = TimeDistributed(MaxPool2D())(op_flow)
    op_flow = TimeDistributed(Conv2D(64, filter_size, activation = "relu", data_format = "channels_last"))(op_flow)
    op_flow = TimeDistributed(Dropout(.3))(op_flow)
    op_flow = TimeDistributed(MaxPool2D())(op_flow)
    op_flow = TimeDistributed(Conv2D(128, filter_size, activation = "relu", data_format = "channels_last"))(op_flow)
    op_flow = TimeDistributed(MaxPool2D())(op_flow)
    op_flow_max = TimeDistributed(GlobalMaxPool2D())(op_flow)
    op_flow_avg = TimeDistributed(GlobalAvgPool2D())(op_flow)
    
    conc=concatenate([op_flow_max,op_flow_avg],axis=1)
    
    conc = SpatialDropout1D(.5)(conc)
    conc = GRU(256)(conc)
    conc = Dense(100, activation = "relu")(conc)
    conc = Dropout(.2)(conc)
    conc = Dense(50, activation = "relu")(conc)
    conc = Dropout(.1)(conc)
    result = Dense(1, activation='linear')(conc)
    
    model = Model(inputs=op_flow_inp, outputs=[result])
    print(model.summary())
    model.compile(loss="mse", optimizer='adam')

    return model
    

In [0]:
model=build_model_history(time_history)
train_indexes, val_indexes=train_test_split(np.arange(int(train_size)), shuffle = True, test_size = .2)

print(train_size,'Training data size per Aug')
print(train_indexes,'Train indices')
print(val_indexes,'Val indices')

maxIndexT=max(train_indexes)
maxIndexV=max(val_indexes)
print(maxIndexT,maxIndexV)

minIndexT=min(train_indexes)
minIndexV=min(val_indexes)
print(minIndexT,minIndexV)

In [0]:
train_generator=DataGenerator(50,hdf5_paths,op_hdf5_paths,FrameIndices,SpeedIndices,train_size,indexes=train_indexes,validation_mode=False)
valid_generator=DataGenerator(50,hdf5_paths,op_hdf5_paths,FrameIndices,SpeedIndices,train_size,indexes=val_indexes,validation_mode=True)
model.fit_generator(train_generator, validation_data=valid_generator, epochs = 10,callbacks=[EarlyStopping(patience=3), ModelCheckpoint(filepath="cnn_time_model_opflow_mid_time_2.h5", save_weights_only=False)])

In [0]:
index_array = np.arange(int(train_size))
split_count = 5
num_epochs = 10
steps_per_epoch = None
kf = KFold(shuffle = False, n_splits = split_count)
custom_splits = list(kf.split(index_array))
for fold in range(split_count):
    train_indexes, val_indexes = custom_splits[fold]
    print("training on fold " + str(fold))
    
    train_generator=DataGenerator(50,hdf5_paths,op_hdf5_paths,FrameIndices,SpeedIndices,train_size,indexes=train_indexes,validation_mode=False)
    valid_generator=DataGenerator(50,hdf5_paths,op_hdf5_paths,FrameIndices,SpeedIndices,train_size,indexes=val_indexes,validation_mode=True)
    model.fit_generator(train_generator, validation_data=valid_generator, epochs = 10,callbacks=[EarlyStopping(patience=3), ModelCheckpoint(filepath="cnn_time_model_opflow_mid_time_2_cross_training.h5", save_weights_only=False)])
    